In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import argparse
import os
import os.path as osp
import shutil
import numpy as np
import pickle
import yaml
from copy import deepcopy
from itertools import repeat
from torch_geometric.data import DataLoader, InMemoryDataset, download_url, extract_zip
from torch_geometric.io import read_tu_data
from torch_geometric.utils import degree
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GINConv, global_add_pool, global_mean_pool, global_max_pool
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

In [2]:
def arg_parse(DS=None):
    parser = argparse.ArgumentParser(description='DGCL Arguments.')

    parser.add_argument('--DS', type=str, default=DS)
    parser.add_argument('--lr', type=float)
    parser.add_argument('--num_gc_layers', type=int)
    parser.add_argument('--hidden_dim', type=int)
    parser.add_argument('--epoch', type=int)
    parser.add_argument('--batch', type=int)
    parser.add_argument('--aug', type=str)
    parser.add_argument('--tau', type=float)
    parser.add_argument('--drop_ratio', type=float)
    parser.add_argument('--num_latent_factors', type=int)
    parser.add_argument('--head_layers', type=int)
    parser.add_argument('--JK', type=str, choices=['last', 'sum'])
    parser.add_argument('--residual', type=int, choices=[0, 1])
    parser.add_argument('--proj', type=int, choices=[0, 1])
    parser.add_argument('--pool', type=str, choices=['mean', 'sum', 'max'])
    parser.add_argument('--fe', type=int)
    parser.add_argument('--seed', type=int)
    parser.add_argument('--num_workers', type=int)
    parser.add_argument('--log_dir', type=str)
    parser.add_argument('--log_interval', type=int, default=5)
    parser.add_argument("--debug", action='store_true', default=False)

    args, unknown = parser.parse_known_args()

    with open(f'config/{args.DS}.yml', 'r') as f:
        config_yaml = yaml.load(f, Loader=yaml.FullLoader)
    for k, v in vars(args).items():
        if v is not None:
            config_yaml[k] = v
    config_ns = argparse.Namespace(**config_yaml)

    return config_ns


def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


def drop_nodes(data):
    node_num, _ = data.x.size()
    _, edge_num = data.edge_index.size()
    drop_num = int(node_num * 0.2)

    idx_drop = np.random.choice(node_num, drop_num, replace=False)
    idx_nondrop = [n for n in range(node_num) if not n in idx_drop]
    idx_dict = {idx_nondrop[n]: n for n in list(range(node_num - drop_num))}

    edge_index = data.edge_index.numpy()

    adj = torch.zeros((node_num, node_num))
    adj[edge_index[0], edge_index[1]] = 1
    adj[idx_drop, :] = 0
    adj[:, idx_drop] = 0
    edge_index = torch.nonzero(adj).t()

    data.edge_index = edge_index
    return data


def permute_edges(data):
    node_num, _ = data.x.size()
    _, edge_num = data.edge_index.size()
    permute_num = int(edge_num * 0.2)

    edge_index = data.edge_index.transpose(0, 1).numpy()

    idx_add = np.random.choice(node_num, (permute_num, 2))

    edge_index = edge_index[np.random.choice(edge_num, edge_num - permute_num, replace=False)]
    data.edge_index = torch.tensor(edge_index).transpose_(0, 1)
    return data


def subgraph(data):
    node_num, _ = data.x.size()
    _, edge_num = data.edge_index.size()
    sub_num = int(node_num * (1 - 0.2))

    edge_index = data.edge_index.numpy()

    idx_sub = [np.random.randint(node_num, size=1)[0]]
    idx_neigh = set([n for n in edge_index[1][edge_index[0] == idx_sub[0]]])

    count = 0
    while len(idx_sub) <= sub_num:
        count = count + 1
        if count > node_num:
            break
        if len(idx_neigh) == 0:
            break
        sample_node = np.random.choice(list(idx_neigh))
        if sample_node in idx_sub:
            continue
        idx_sub.append(sample_node)
        idx_neigh.union(set([n for n in edge_index[1][edge_index[0] == idx_sub[-1]]]))

    idx_drop = [n for n in range(node_num) if not n in idx_sub]
    idx_nondrop = idx_sub
    idx_dict = {idx_nondrop[n]: n for n in list(range(len(idx_nondrop)))}

    edge_index = data.edge_index.numpy()

    adj = torch.zeros((node_num, node_num))
    adj[edge_index[0], edge_index[1]] = 1
    adj[idx_drop, :] = 0
    adj[:, idx_drop] = 0
    edge_index = torch.nonzero(adj).t()

    data.edge_index = edge_index
    return data


def mask_nodes(data):
    node_num, feat_dim = data.x.size()
    mask_num = int(node_num * 0.2)

    idx_mask = np.random.choice(node_num, mask_num, replace=False)
    data.x[idx_mask] = torch.tensor(np.random.normal(loc=0.5, scale=0.5, size=(mask_num, feat_dim)),
                                    dtype=torch.float32)

    return data


def svc_classify(x, y, search):
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    accuracies = []
    accuracies_val = []
    for train_index, test_index in kf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if search:
            params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
            classifier = GridSearchCV(SVC(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = SVC(C=10)
        classifier.fit(x_train, y_train)
        accuracies.append(accuracy_score(y_test, classifier.predict(x_test)))

        val_size = len(test_index)
        test_index = np.random.choice(train_index, val_size, replace=False).tolist()
        train_index = [i for i in train_index if not i in test_index]

        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if search:
            params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
            classifier = GridSearchCV(SVC(), params, cv=5, scoring='accuracy', verbose=0)
        else:
            classifier = SVC(C=10)
        classifier.fit(x_train, y_train)
        accuracies_val.append(accuracy_score(y_test, classifier.predict(x_test)))

    return np.mean(accuracies_val), np.mean(accuracies)


def evaluate_embedding(embeddings, labels, search=True):
    labels = preprocessing.LabelEncoder().fit_transform(labels)
    x, y = np.array(embeddings), np.array(labels)
    acc_val, acc = svc_classify(x, y, search)
    return acc, acc_val


class DGCL(nn.Module):
    def __init__(self, num_features, hidden_dim, num_layer, device, args):
        super(DGCL, self).__init__()

        self.args = args
        self.device = device
        self.T = self.args.tau
        self.K = args.num_latent_factors
        self.embedding_dim = hidden_dim
        self.d = self.embedding_dim // self.K

        self.center_v = torch.rand((self.K, self.d), requires_grad=True).to(device)

        self.encoder = DisenEncoder(
            num_features=num_features,
            emb_dim=hidden_dim,
            num_layer=num_layer,
            K=args.num_latent_factors,
            head_layers=args.head_layers,
            device=device,
            args=args,
            if_proj_head=args.proj > 0,
            drop_ratio=args.drop_ratio,
            graph_pooling=args.pool,
            JK=args.JK,
            residual=args.residual > 0
        )

        self.init_emb()

    def init_emb(self):
        initrange = -1.5 / self.embedding_dim
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.0)

    def forward(self, x, edge_index, batch, num_graphs):
        if x is None:
            x = torch.ones(batch.shape[0]).to(device)
        z_graph, _ = self.encoder(x, edge_index, batch)
        return z_graph

    def loss_cal(self, x, x_aug):
        T = self.T
        T_c = 0.2
        B, H, d = x.size()
        ck = F.normalize(self.center_v)
        p_k_x_ = torch.einsum('bkd,kd->bk', F.normalize(x, dim=-1), ck)
        p_k_x = F.softmax(p_k_x_ / T_c, dim=-1)
        x_abs = x.norm(dim=-1)
        x_aug_abs = x_aug.norm(dim=-1)
        x = torch.reshape(x, (B * H, d))
        x_aug = torch.reshape(x_aug, (B * H, d))
        x_abs = torch.squeeze(torch.reshape(x_abs, (B * H, 1)), 1)
        x_aug_abs = torch.squeeze(torch.reshape(x_aug_abs, (B * H, 1)), 1)
        sim_matrix = torch.einsum('ik,jk->ij', x, x_aug) / (1e-8 + torch.einsum('i,j->ij', x_abs, x_aug_abs))
        sim_matrix = torch.exp(sim_matrix / T)
        pos_sim = sim_matrix[range(B * H), range(B * H)]
        score = pos_sim / (sim_matrix.sum(dim=-1) - pos_sim)
        p_y_xk = score.view(B, H)
        q_k = torch.einsum('bk,bk->bk', p_k_x, p_y_xk)
        q_k = F.normalize(q_k, dim=-1)
        elbo = q_k * (torch.log(p_k_x) + torch.log(p_y_xk) - torch.log(q_k))
        loss = - elbo.view(-1).mean()
        return loss


class DisenEncoder(torch.nn.Module):
    def __init__(self, num_features, emb_dim, num_layer, K, head_layers, if_proj_head=False, drop_ratio=0.0,
                 graph_pooling='add', JK='last', residual=False, device=None, args=None):
        super(DisenEncoder, self).__init__()
        self.args = args
        self.device = device
        self.num_features = num_features
        self.K = K
        self.d = emb_dim // self.K
        self.num_layer = num_layer
        self.head_layers = head_layers
        self.gc_layers = self.num_layer - self.head_layers
        self.if_proj_head = if_proj_head
        self.drop_ratio = drop_ratio
        self.graph_pooling = graph_pooling
        if self.graph_pooling == "sum" or self.graph_pooling == 'add':
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        else:
            raise ValueError("Invalid graph pooling type.")
        self.JK = JK
        if JK == 'last':
            pass
        elif JK == 'sum':
            self.JK_proj = Linear(self.gc_layers * emb_dim, emb_dim)
        else:
            assert False
        self.residual = residual
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        self.disen_convs = torch.nn.ModuleList()
        self.disen_bns = torch.nn.ModuleList()

        for i in range(self.gc_layers):
            if i == 0:
                nn = Sequential(Linear(num_features, emb_dim), ReLU(), Linear(emb_dim, emb_dim))
            else:
                nn = Sequential(Linear(emb_dim, emb_dim), ReLU(), Linear(emb_dim, emb_dim))
            conv = GINConv(nn)
            bn = torch.nn.BatchNorm1d(emb_dim)

            self.convs.append(conv)
            self.bns.append(bn)

        for i in range(self.K):
            for j in range(self.head_layers):
                if j == 0:
                    nn = Sequential(Linear(emb_dim, self.d), ReLU(), Linear(self.d, self.d))
                else:
                    nn = Sequential(Linear(self.d, self.d), ReLU(), Linear(self.d, self.d))
                conv = GINConv(nn)
                bn = torch.nn.BatchNorm1d(self.d)

                self.disen_convs.append(conv)
                self.disen_bns.append(bn)

        self.proj_heads = torch.nn.ModuleList()
        for i in range(self.K):
            nn = Sequential(Linear(self.d, self.d), ReLU(inplace=True), Linear(self.d, self.d))
            self.proj_heads.append(nn)

    def _normal_conv(self, x, edge_index, batch):
        xs = []
        for i in range(self.gc_layers):
            x = self.convs[i](x, edge_index)
            x = self.bns[i](x)
            if i == self.gc_layers - 1:
                x = F.dropout(x, self.drop_ratio, training=self.training)
            else:
                x = F.dropout(F.relu(x), self.drop_ratio, training=self.training)

            if self.residual and i > 0:
                x += xs[i - 1]
            xs.append(x)
        if self.JK == 'last':
            return xs[-1]
        elif self.JK == 'sum':
            return self.JK_proj(torch.cat(xs, dim=-1))

    def _disen_conv(self, x, edge_index, batch):
        x_proj_list = []
        x_proj_pool_list = []
        for i in range(self.K):
            x_proj = x
            for j in range(self.head_layers):
                tmp_index = i * self.head_layers + j
                x_proj = self.disen_convs[tmp_index](x_proj, edge_index)
                x_proj = self.disen_bns[tmp_index](x_proj)
                if j != self.head_layers - 1:
                    x_proj = F.relu(x_proj)
            x_proj_list.append(x_proj)
            x_proj_pool_list.append(self.pool(x_proj, batch))
        if self.if_proj_head:
            x_proj_pool_list = self._proj_head(x_proj_pool_list)
        x_graph_multi = torch.stack(x_proj_pool_list)
        x_node_multi = torch.stack(x_proj_list)
        x_graph_multi = x_graph_multi.permute(1, 0, 2).contiguous()
        x_node_multi = x_node_multi.permute(1, 0, 2).contiguous()
        return x_graph_multi, x_node_multi

    def _proj_head(self, x_proj_pool_list):
        ret = []
        for k in range(self.K):
            x_graph_proj = self.proj_heads[k](x_proj_pool_list[k])
            ret.append(x_graph_proj)
        return ret

    def forward(self, x, edge_index, batch):
        if x is None:
            x = torch.ones((batch.shape[0], 1)).to(device)
        h_node = self._normal_conv(x, edge_index, batch)
        h_graph_multi, h_node_multi = self._disen_conv(h_node, edge_index, batch)
        return h_graph_multi, h_node_multi

    def get_embeddings(self, loader):
        device = self.device
        ret = []
        y = []
        with torch.no_grad():
            for data in loader:
                data = data[0]
                data.to(device)
                x, edge_index, batch = data.x, data.edge_index, data.batch
                if x is None:
                    x = torch.ones((batch.shape[0], 1)).to(device)
                x, _ = self.forward(x, edge_index, batch)
                B, K, d = x.size()
                x = x.view(B, K * d)
                ret.append(x.cpu().numpy())
                y.append(data.y.cpu().numpy())
        ret = np.concatenate(ret, 0)
        y = np.concatenate(y, 0)
        return ret, y


class TUDataset(InMemoryDataset):
    url = ('http://ls11-www.cs.tu-dortmund.de/people/morris/'
           'graphkerneldatasets')
    cleaned_url = ('https://raw.githubusercontent.com/nd7141/'
                   'graph_datasets/master/datasets')

    def __init__(self, root, name, transform=None, pre_transform=None,
                 pre_filter=None, use_node_attr=False, use_edge_attr=False,
                 cleaned=False, aug=None, args=None):
        self.name = name
        self.cleaned = cleaned

        super(TUDataset, self).__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        if self.data.x is not None and not use_node_attr:
            num_node_attributes = self.num_node_attributes
            self.data.x = self.data.x[:, num_node_attributes:]
        if self.data.edge_attr is not None and not use_edge_attr:
            num_edge_attributes = self.num_edge_attributes
            self.data.edge_attr = self.data.edge_attr[:, num_edge_attributes:]
        if not (self.name == 'MUTAG' or self.name == 'PTC_MR' or self.name == 'DD' or self.name == 'PROTEINS' or self.name == 'NCI1' or self.name == 'NCI109'):
            edge_index = self.data.edge_index[0, :].numpy()
            _, num_edge = self.data.edge_index.size()
            nlist = [edge_index[n] + 1 for n in range(num_edge - 1) if edge_index[n] > edge_index[n + 1]]
            nlist.append(edge_index[-1] + 1)

            num_node = np.array(nlist).sum()

            if args.fe == 0:
                self.data.x = torch.ones((num_node, 1))
            elif args.fe == 1:
                deg_file = os.path.join(root, name, 'degree.pickle')
                if not os.path.exists(deg_file):
                    edge_nums = []
                    for i, _ in enumerate(self.slices['edge_index']):
                        if i:
                            edge_nums.append(self.slices['edge_index'][i] - self.slices['edge_index'][i - 1])
                    edge_indexs = torch.split(self.data.edge_index[1], edge_nums)
                    degs_2d = [degree(edge_index, nlist[i], dtype=torch.long) for i, edge_index in
                               enumerate(edge_indexs)]
                    degs_1d = []
                    for i in degs_2d:
                        for j in i:
                            degs_1d.append(j)

                    max_deg = int(max(degs_1d))
                    degs_1d = torch.stack(degs_1d)
                    deg_f = F.one_hot(degs_1d, num_classes=max_deg + 1).to(torch.float)
                    if not os.path.exists(deg_file):
                        with open(deg_file, 'wb') as handle:
                            pickle.dump(deg_f, handle, protocol=pickle.HIGHEST_PROTOCOL)
                else:
                    with open(deg_file, 'rb') as handle:
                        deg_f = pickle.load(handle)
                self.data.x = deg_f

            edge_slice = [0]
            k = 0
            for n in nlist:
                k = k + n
                edge_slice.append(k)
            self.slices['x'] = torch.tensor(edge_slice)
        self.aug = aug

    @property
    def raw_dir(self):
        name = 'raw{}'.format('_cleaned' if self.cleaned else '')
        return osp.join(self.root, self.name, name)

    @property
    def processed_dir(self):
        name = 'processed{}'.format('_cleaned' if self.cleaned else '')
        return osp.join(self.root, self.name, name)

    @property
    def num_node_labels(self):
        if self.data.x is None:
            return 0
        for i in range(self.data.x.size(1)):
            x = self.data.x[:, i:]
            if ((x == 0) | (x == 1)).all() and (x.sum(dim=1) == 1).all():
                return self.data.x.size(1) - i
        return 0

    @property
    def num_node_attributes(self):
        if self.data.x is None:
            return 0
        return self.data.x.size(1) - self.num_node_labels

    @property
    def num_edge_labels(self):
        if self.data.edge_attr is None:
            return 0
        for i in range(self.data.edge_attr.size(1)):
            if self.data.edge_attr[:, i:].sum() == self.data.edge_attr.size(0):
                return self.data.edge_attr.size(1) - i
        return 0

    @property
    def num_edge_attributes(self):
        if self.data.edge_attr is None:
            return 0
        return self.data.edge_attr.size(1) - self.num_edge_labels

    @property
    def raw_file_names(self):
        names = ['A', 'graph_indicator']
        return ['{}_{}.txt'.format(self.name, name) for name in names]

    @property
    def processed_file_names(self):
        return 'data.pt'

    def download(self):
        url = self.cleaned_url if self.cleaned else self.url
        folder = osp.join(self.root, self.name)
        path = download_url('{}/{}.zip'.format(url, self.name), folder)
        extract_zip(path, folder)
        os.unlink(path)
        shutil.rmtree(self.raw_dir)
        os.rename(osp.join(folder, self.name), self.raw_dir)

    def process(self):
        self.data, self.slices = read_tu_data(self.raw_dir, self.name)

        if self.pre_filter is not None:
            data_list = [self.get(idx) for idx in range(len(self))]
            data_list = [data for data in data_list if self.pre_filter(data)]
            self.data, self.slices = self.collate(data_list)

        if self.pre_transform is not None:
            data_list = [self.get(idx) for idx in range(len(self))]
            data_list = [self.pre_transform(data) for data in data_list]
            self.data, self.slices = self.collate(data_list)

        torch.save((self.data, self.slices), self.processed_paths[0])

    def __repr__(self):
        return '{}({})'.format(self.name, len(self))

    def get_num_feature(self):
        data = self.data.__class__()

        if hasattr(self.data, '__num_nodes__'):
            data.num_nodes = self.data.__num_nodes__[0]

        for key in self.data.keys:
            try:
                item, slices = self.data[key], self.slices[key]
            except:
                continue

            if torch.is_tensor(item):
                s = list(repeat(slice(None), item.dim()))
                s[self.data.__cat_dim__(key,
                                        item)] = slice(slices[0],
                                                       slices[0 + 1])
            else:
                s = slice(slices[idx], slices[idx + 1])
            data[key] = item[s]
        _, num_feature = data.x.size()

        return num_feature

    def get(self, idx):
        data = self.data.__class__()

        if hasattr(self.data, '__num_nodes__'):
            data.num_nodes = self.data.__num_nodes__[idx]

        for key in self.data.keys:
            try:
                item, slices = self.data[key], self.slices[key]
            except:
                continue
            if torch.is_tensor(item):
                s = list(repeat(slice(None), item.dim()))
                s[self.data.__cat_dim__(key,
                                        item)] = slice(slices[idx],
                                                       slices[idx + 1])
            else:
                s = slice(slices[idx], slices[idx + 1])
            data[key] = item[s]

        node_num = data.edge_index.max()
        sl = torch.tensor([[n, n] for n in range(node_num)]).t()
        data.edge_index = torch.cat((data.edge_index, sl), dim=1)

        if self.aug == 'dnodes':
            data_aug = drop_nodes(deepcopy(data))
        elif self.aug == 'pedges':
            data_aug = permute_edges(deepcopy(data))
        elif self.aug == 'subgraph':
            data_aug = subgraph(deepcopy(data))
        elif self.aug == 'mask_nodes':
            data_aug = mask_nodes(deepcopy(data))
        elif self.aug == 'none':
            data_aug = deepcopy(data)
            data_aug.x = torch.ones((data.edge_index.max() + 1, 1))
        elif self.aug == 'random4':
            n = np.random.randint(4)
            if n == 0:
                data_aug = drop_nodes(deepcopy(data))
            elif n == 1:
                data_aug = permute_edges(deepcopy(data))
            elif n == 2:
                data_aug = subgraph(deepcopy(data))
            elif n == 3:
                data_aug = mask_nodes(deepcopy(data))
            else:
                print('sample error')
                assert False
        else:
            print('augmentation error')
            assert False

        return data, data_aug




In [3]:
args = arg_parse('MUTAG')
setup_seed(args.seed)

accuracies = {'result': [], 'result_val': []}
epochs = args.epoch
log_interval = args.log_interval
batch_size = args.batch
lr = args.lr
DS = args.DS
args.hidden_dim = int(args.hidden_dim // args.num_latent_factors) * args.num_latent_factors
path = './data'
dataset = TUDataset(path, name=DS, aug=args.aug, args=args).shuffle()
dataset_eval = TUDataset(path, name=DS, aug='none', args=args).shuffle()
print(len(dataset))
print(dataset.get_num_feature())
try:
    dataset_num_features = dataset.get_num_feature()
except:
    dataset_num_features = 1
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=args.num_workers)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, num_workers=args.num_workers)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCL(
    num_features=dataset_num_features,
    hidden_dim=args.hidden_dim,
    num_layer=args.num_gc_layers,
    device=device,
    args=args
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.eval()
emb, y = model.encoder.get_embeddings(dataloader_eval)
for epoch in range(1, epochs + 1):
    loss_all = 0
    model.train()
    for data in dataloader:
        data, data_aug = data
        optimizer.zero_grad()
        node_num, _ = data.x.size()
        data = data.to(device)
        x = model(data.x, data.edge_index, data.batch, data.num_graphs)
        edge_idx = data_aug.edge_index.numpy()
        _, edge_num = edge_idx.shape
        idx_not_missing = [n for n in range(node_num) if (n in edge_idx[0] or n in edge_idx[1])]
        node_num_aug = len(idx_not_missing)
        data_aug.x = data_aug.x[idx_not_missing]
        data_aug.batch = data.batch[idx_not_missing]
        idx_dict = {idx_not_missing[n]: n for n in range(node_num_aug)}
        edge_idx = [[idx_dict[edge_idx[0, n]], idx_dict[edge_idx[1, n]]] for n in range(edge_num) if
                    not edge_idx[0, n] == edge_idx[1, n]]
        data_aug.edge_index = torch.tensor(edge_idx).transpose_(0, 1)
        data_aug = data_aug.to(device)
        x_aug = model(data_aug.x, data_aug.edge_index, data_aug.batch, data_aug.num_graphs)
        loss = model.loss_cal(x, x_aug)
        loss_all += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    print('loss %.4f' % (loss_all / len(dataloader)))
    if log_interval > 0 and epoch % log_interval == 0:
        model.eval()
        emb, y = model.encoder.get_embeddings(dataloader_eval)
        result, result_val = evaluate_embedding(emb, y)
        accuracies['result'].append(result)
        accuracies['result_val'].append(result_val)

print(accuracies)

188
7
loss 241.1538
loss 232.5214
loss 230.3310
loss 230.5811
loss 226.9621
loss 233.5162
loss 232.8569
loss 230.5816
loss 227.8818
loss 216.3842
loss 218.7928
loss 214.4476
loss 208.7943
loss 213.3354
loss 206.4141
loss 208.4692
loss 212.2978
loss 199.6359
loss 207.7666
loss 195.3267
loss 196.3122
loss 196.5613
loss 192.0900
loss 190.9878
loss 189.9690
loss 184.1631
loss 182.8659
loss 179.8863
loss 180.6454
loss 181.1206
{'result': [0.8994152046783626, 0.8614035087719298, 0.9099415204678362, 0.9201754385964913, 0.9248538011695906, 0.9365497076023391], 'result_val': [0.8669590643274854, 0.8780701754385966, 0.9149122807017545, 0.9257309941520468, 0.9412280701754385, 0.9152046783625731]}


In [4]:
args = arg_parse('PTC_MR')
setup_seed(args.seed)

accuracies = {'result': [], 'result_val': []}
epochs = args.epoch
log_interval = args.log_interval
batch_size = args.batch
lr = args.lr
DS = args.DS
args.hidden_dim = int(args.hidden_dim // args.num_latent_factors) * args.num_latent_factors
path = './data'
dataset = TUDataset(path, name=DS, aug=args.aug, args=args).shuffle()
dataset_eval = TUDataset(path, name=DS, aug='none', args=args).shuffle()
print(len(dataset))
print(dataset.get_num_feature())
try:
    dataset_num_features = dataset.get_num_feature()
except:
    dataset_num_features = 1
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=args.num_workers)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, num_workers=args.num_workers)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCL(
    num_features=dataset_num_features,
    hidden_dim=args.hidden_dim,
    num_layer=args.num_gc_layers,
    device=device,
    args=args
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.eval()
emb, y = model.encoder.get_embeddings(dataloader_eval)
for epoch in range(1, epochs + 1):
    loss_all = 0
    model.train()
    for data in dataloader:
        data, data_aug = data
        optimizer.zero_grad()
        node_num, _ = data.x.size()
        data = data.to(device)
        x = model(data.x, data.edge_index, data.batch, data.num_graphs)
        edge_idx = data_aug.edge_index.numpy()
        _, edge_num = edge_idx.shape
        idx_not_missing = [n for n in range(node_num) if (n in edge_idx[0] or n in edge_idx[1])]
        node_num_aug = len(idx_not_missing)
        data_aug.x = data_aug.x[idx_not_missing]
        data_aug.batch = data.batch[idx_not_missing]
        idx_dict = {idx_not_missing[n]: n for n in range(node_num_aug)}
        edge_idx = [[idx_dict[edge_idx[0, n]], idx_dict[edge_idx[1, n]]] for n in range(edge_num) if
                    not edge_idx[0, n] == edge_idx[1, n]]
        data_aug.edge_index = torch.tensor(edge_idx).transpose_(0, 1)
        data_aug = data_aug.to(device)
        x_aug = model(data_aug.x, data_aug.edge_index, data_aug.batch, data_aug.num_graphs)
        loss = model.loss_cal(x, x_aug)
        loss_all += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    print('loss %.4f' % (loss_all / len(dataloader)))
    if log_interval > 0 and epoch % log_interval == 0:
        model.eval()
        emb, y = model.encoder.get_embeddings(dataloader_eval)
        result, result_val = evaluate_embedding(emb, y)
        accuracies['result'].append(result)
        accuracies['result_val'].append(result_val)

print(accuracies)

344
18
loss 124.1277
loss 123.2824
loss 116.5900
loss 116.6115
loss 115.1440
loss 110.8785
loss 107.3974
loss 108.0987
loss 104.3184
loss 101.3686
loss 100.1917
loss 100.8181
loss 96.7437
loss 94.8441
loss 91.9972
loss 92.5601
loss 89.2327
loss 88.4863
loss 87.1843
loss 86.2084
{'result': [0.6132773109243698, 0.5991596638655463, 0.5923529411764706, 0.6742016806722688], 'result_val': [0.5491596638655463, 0.5931932773109244, 0.5990756302521009, 0.6717647058823528]}


In [5]:
args = arg_parse('PROTEINS')
setup_seed(args.seed)

accuracies = {'result': [], 'result_val': []}
epochs = args.epoch
log_interval = args.log_interval
batch_size = args.batch
lr = args.lr
DS = args.DS
args.hidden_dim = int(args.hidden_dim // args.num_latent_factors) * args.num_latent_factors
path = './data'
dataset = TUDataset(path, name=DS, aug=args.aug, args=args).shuffle()
dataset_eval = TUDataset(path, name=DS, aug='none', args=args).shuffle()
print(len(dataset))
print(dataset.get_num_feature())
try:
    dataset_num_features = dataset.get_num_feature()
except:
    dataset_num_features = 1
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=args.num_workers)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, num_workers=args.num_workers)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCL(
    num_features=dataset_num_features,
    hidden_dim=args.hidden_dim,
    num_layer=args.num_gc_layers,
    device=device,
    args=args
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.eval()
emb, y = model.encoder.get_embeddings(dataloader_eval)
for epoch in range(1, epochs + 1):
    loss_all = 0
    model.train()
    for data in dataloader:
        data, data_aug = data
        optimizer.zero_grad()
        node_num, _ = data.x.size()
        data = data.to(device)
        x = model(data.x, data.edge_index, data.batch, data.num_graphs)
        edge_idx = data_aug.edge_index.numpy()
        _, edge_num = edge_idx.shape
        idx_not_missing = [n for n in range(node_num) if (n in edge_idx[0] or n in edge_idx[1])]
        node_num_aug = len(idx_not_missing)
        data_aug.x = data_aug.x[idx_not_missing]
        data_aug.batch = data.batch[idx_not_missing]
        idx_dict = {idx_not_missing[n]: n for n in range(node_num_aug)}
        edge_idx = [[idx_dict[edge_idx[0, n]], idx_dict[edge_idx[1, n]]] for n in range(edge_num) if
                    not edge_idx[0, n] == edge_idx[1, n]]
        data_aug.edge_index = torch.tensor(edge_idx).transpose_(0, 1)
        data_aug = data_aug.to(device)
        x_aug = model(data_aug.x, data_aug.edge_index, data_aug.batch, data_aug.num_graphs)
        loss = model.loss_cal(x, x_aug)
        loss_all += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    print('loss %.4f' % (loss_all / len(dataloader)))
    if log_interval > 0 and epoch % log_interval == 0:
        model.eval()
        emb, y = model.encoder.get_embeddings(dataloader_eval)
        result, result_val = evaluate_embedding(emb, y)
        accuracies['result'].append(result)
        accuracies['result_val'].append(result_val)

print(accuracies)

1113
3
loss 474.3534
loss 330.3853
loss 270.1005
loss 242.7528
loss 226.5136
loss 220.2125
loss 214.8679
loss 210.2680
loss 207.8771
loss 206.2781
loss 203.7304
loss 204.1883
loss 201.3004
loss 200.8594
loss 197.6796
loss 197.2809
loss 195.9566
loss 196.1921
loss 194.1449
loss 191.6807
{'result': [0.7520592020592021, 0.7583172458172458, 0.7564913127413126, 0.7691200128700129], 'result_val': [0.7583976833976834, 0.7591618404118405, 0.7511422136422137, 0.7501287001287]}


In [6]:
args = arg_parse('IMDB-MULTI')
setup_seed(args.seed)

accuracies = {'result': [], 'result_val': []}
epochs = args.epoch
log_interval = args.log_interval
batch_size = args.batch
lr = args.lr
DS = args.DS
args.hidden_dim = int(args.hidden_dim // args.num_latent_factors) * args.num_latent_factors
path = './data'
dataset = TUDataset(path, name=DS, aug=args.aug, args=args).shuffle()
dataset_eval = TUDataset(path, name=DS, aug='none', args=args).shuffle()
print(len(dataset))
print(dataset.get_num_feature())
try:
    dataset_num_features = dataset.get_num_feature()
except:
    dataset_num_features = 1
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=args.num_workers)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, num_workers=args.num_workers)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCL(
    num_features=dataset_num_features,
    hidden_dim=args.hidden_dim,
    num_layer=args.num_gc_layers,
    device=device,
    args=args
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.eval()
emb, y = model.encoder.get_embeddings(dataloader_eval)
for epoch in range(1, epochs + 1):
    loss_all = 0
    model.train()
    for data in dataloader:
        data, data_aug = data
        optimizer.zero_grad()
        node_num, _ = data.x.size()
        data = data.to(device)
        x = model(data.x, data.edge_index, data.batch, data.num_graphs)
        edge_idx = data_aug.edge_index.numpy()
        _, edge_num = edge_idx.shape
        idx_not_missing = [n for n in range(node_num) if (n in edge_idx[0] or n in edge_idx[1])]
        node_num_aug = len(idx_not_missing)
        data_aug.x = data_aug.x[idx_not_missing]
        data_aug.batch = data.batch[idx_not_missing]
        idx_dict = {idx_not_missing[n]: n for n in range(node_num_aug)}
        edge_idx = [[idx_dict[edge_idx[0, n]], idx_dict[edge_idx[1, n]]] for n in range(edge_num) if
                    not edge_idx[0, n] == edge_idx[1, n]]
        data_aug.edge_index = torch.tensor(edge_idx).transpose_(0, 1)
        data_aug = data_aug.to(device)
        x_aug = model(data_aug.x, data_aug.edge_index, data_aug.batch, data_aug.num_graphs)
        loss = model.loss_cal(x, x_aug)
        loss_all += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    print('loss %.4f' % (loss_all / len(dataloader)))
    if log_interval > 0 and epoch % log_interval == 0:
        model.eval()
        emb, y = model.encoder.get_embeddings(dataloader_eval)
        result, result_val = evaluate_embedding(emb, y)
        accuracies['result'].append(result)
        accuracies['result_val'].append(result_val)

print(accuracies)

1500
89
loss 840.7354
loss 530.6607
loss 480.1475
loss 457.7489
loss 435.8316
loss 427.8606
loss 415.7121
loss 411.9177
loss 405.0758
loss 405.1614
{'result': [0.514, 0.53], 'result_val': [0.514, 0.4833333333333333]}
